<a href="https://colab.research.google.com/github/Prathamesh-kadam/GNN-Model/blob/main/Accessing_the_data_from_website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import requests # Import the requests library

# Define the URL for the file
url = 'https://lweb.cfa.harvard.edu/supernova/CfA3/cfa3lightcurves.standardsystem.txt'
filename = 'cfa3lightcurves.standardsystem.txt'

# Download the file if it doesn't exist
try:
    with open(filename, 'r') as f:
        pass # File exists, do nothing
except FileNotFoundError:
    print(f'Downloading {filename}...')
    response = requests.get(url)
    response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f'Successfully downloaded {filename}.')

# Read the downloaded file
df_raw = pd.read_csv(filename,
                     sep=r'\s+', comment='#', header=None,
                     names=['col1', 'col2', 'col3', 'col4'])

# Filter out non-data rows and parse properly
data = []
current_sn = None
filter_map = {1: 'U', 2: 'B', 3: 'V', 4: 'R', 5: 'I', 13: "r'", 14: "i'"}

for index, row in df_raw.iterrows():
    # Check if this is an SN name (string starting with 'sn')
    if pd.isna(row['col1']) == False and str(row['col1']).startswith('sn'):
        current_sn = str(row['col1'])
        continue

    # Check if this is a data row (filter code as first column)
    if (pd.notna(row['col1']) and str(row['col1']).isdigit() and
        int(row['col1']) in filter_map):

        filt = int(row['col1'])
        mjd = float(row['col2'])
        stdmag = float(row['col3'])
        dmag = float(row['col4'])

        data.append({
            'SN': current_sn,
            'MJD': round(mjd, 5),  # Round for cleaner display
            'Filter': filter_map[filt],
            'STDMAG': stdmag,
            'dMAG': dmag
        })

# Create clean DataFrame
df = pd.DataFrame(data)

# Sort by SN, then MJD, then Filter for proper grouping
df = df.sort_values(['SN', 'MJD', 'Filter']).reset_index(drop=True)

# Save to Excel and CSV
df.to_excel('cfa3_lightcurves.xlsx', index=False)
df.to_csv('cfa3_lightcurves.csv', index=False)

# Display summary
print(f"✅ Success! Processed {len(df)} observations from {df['SN'].nunique()} supernovae")
print("\nFirst 10 rows:")
print(df.head(10))
print("\nColumn structure:")
print(df.dtypes)
print("\nUnique SNe:", sorted(df['SN'].unique())[:10], "...")

Successfully downloaded cfa3lightcurves.standardsystem.txt.
✅ Success! Processed 11499 observations from 185 supernovae

First 10 rows:
      SN          MJD Filter  STDMAG   dMAG
0  sn01C  51933.28865      U  17.043  0.035
1  sn01C  51933.29189      B  16.503  0.015
2  sn01C  51933.29438      V  15.442  0.014
3  sn01C  51933.29616      R  15.170  0.019
4  sn01C  51933.29758      I  15.181  0.022
5  sn01C  51957.12972      U  18.000  0.039
6  sn01C  51957.13304      B  17.639  0.027
7  sn01C  51957.13560      V  16.557  0.022
8  sn01C  51957.13747      R  16.177  0.028
9  sn01C  51963.19172      U  18.111  0.069

Column structure:
SN         object
MJD       float64
Filter     object
STDMAG    float64
dMAG      float64
dtype: object

Unique SNe: ['sn01C', 'sn01G', 'sn01N', 'sn01V', 'sn01ah', 'sn01ay', 'sn01az', 'sn01bf', 'sn01cp', 'sn01da'] ...


In [ ]:
from google.colab import files

files.download('cfa3_lightcurves.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(f"Number of unique supernovae: {df['SN'].nunique()}")

Number of unique supernovae: 185


The below code is to determine the apparent magnitude from the standard magnitude abotained for our 40 shortlisted supernovae matching it with 185 supernovae


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
from astropy.stats import sigma_clipped_stats
import warnings
warnings.filterwarnings('ignore')

# File paths
shortlisted_path = '/content/drive/MyDrive/apparent magnitude data/updated 40s_dataset.xlsx'  # Your 40 SNe [file:42]
cfa_path = '/content/drive/MyDrive/apparent magnitude data/cfa3_lightcurves (1).xlsx'          # CFA3 lightcurves [file:41]
output_path = '/content/drive/MyDrive/apparent magnitude data/updated_40s_with_peak_magnitudes.xlsx' # Changed output filename


print("🔄 Loading datasets...")
short_df = pd.read_excel(shortlisted_path, sheet_name='Sheet1')
cfa_df = pd.read_excel(cfa_path, sheet_name='Sheet1')

print(f"✅ Shortlisted: {len(short_df)} SNe")
print(f"✅ CFA3: {len(cfa_df)} data points")
print("CFA3 columns:", cfa_df.columns.tolist())

sn_names = short_df.iloc[:, 0].astype(str).str.strip().tolist()
print(f"First 5 shortlisted SNe: {sn_names[:5]}")

def extract_peak_magnitude(jd, mag, mag_err, n_iter=10000):
    """YOUR EXACT ORIGINAL CODE - unchanged"""
    if len(jd) < 5:
        return np.nan, np.nan

    try:
        # --- Fit initial spline ---
        spline = UnivariateSpline(jd, mag, k=4, s=0.1)
        fine_jd = np.linspace(jd.min(), jd.max(), 1000)
        fine_mag = spline(fine_jd)

        # --- Monte Carlo Simulation ---
        mag_max_sim = []
        for i in range(n_iter):
            mag_sim = mag + np.random.normal(0, mag_err)
            spline_sim = UnivariateSpline(jd, mag_sim, k=4, s=0.1)
            fine_mag_sim = spline_sim(fine_jd)
            mag_max_sim.append(np.min(fine_mag_sim))

        # --- Sigma-Clipped Stats ---
        mag_max_mean, _, mag_max_std = sigma_clipped_stats(np.array(mag_max_sim))
        return mag_max_mean, mag_max_std

    except:
        return np.nan, np.nan

# =============================================================================
# PERFECT SN NAME MATCHING - Handles ALL CFA3 formats
# =============================================================================
def match_sn_names(short_name, cfa_names):
    """Matches: 1986G→SN1986G/SN86G, 2001cp→SN2001cp/SN01cp"""
    short_name = short_name.strip().upper()

    # Extract year components
    if len(short_name) >= 5 and short_name[:4].isdigit():
        year_full = short_name[:4]  # 1986, 2001
        year_short = year_full[2:]  # 86, 01
        sn_id = short_name[4:]      # G, cp
    else:
        year_full, year_short, sn_id = short_name, short_name, ''

    candidates = []

    for cfa_name in cfa_names:
        cfa_clean = str(cfa_name).strip().upper()

        # Pattern 1: Exact match
        if short_name == cfa_clean:
            candidates.append(cfa_clean)

        # Pattern 2: SN + FULL YEAR (SN1986G, SN2001cp)
        if f'SN{year_full}{sn_id}' in cfa_clean:
            candidates.append(cfa_clean)

        # Pattern 3: SN + SHORT YEAR (SN86G, SN01cp) - CRITICAL!
        if f'SN{year_short}{sn_id}' in cfa_clean:
            candidates.append(cfa_clean)

    return candidates[0] if candidates else None

print("\n🚀 Extracting peak magnitudes for ALL FILTERS...")
results_df = short_df.copy()

# CFA3 filter-specific columns
filter_columns = {
    'U': {'mag': 'STDMAG_U', 'err': 'dMAG_U'},
    'B': {'mag': 'STDMAG_B', 'err': 'dMAG_B'},
    'V': {'mag': 'STDMAG_V', 'err': 'dMAG_V'},
    'R': {'mag': 'STDMAG_R', 'err': 'dMAG_R'},
    'I': {'mag': 'STDMAG_I', 'err': 'dMAG_I'}
}

# Add result columns for each filter
for f in filter_columns.keys():
    results_df[f'{f}_m_max'] = np.nan
    results_df[f'{f}_m_max_err'] = np.nan

# =============================================================================
# MAIN PROCESSING LOOP
# =============================================================================
successful_matches = 0
for idx, short_sn in enumerate(sn_names):
    print(f"\n[{idx+1}/{len(sn_names)}] Processing: {short_sn}")

    # Find CFA3 match
    cfa_sn_name = match_sn_names(short_sn, cfa_df['SN'].unique())

    if cfa_sn_name is None:
        print(f"  ❌ No CFA3 match")
        continue

    print(f"  ✅ MATCHED: {short_sn} → {cfa_sn_name}")
    successful_matches += 1

    # Get CFA3 data for this SN
    sn_mask = cfa_df['SN'].astype(str).str.strip().str.upper() == cfa_sn_name
    sn_data = cfa_df[sn_mask]

    print(f"  📊 {len(sn_data)} data points")

    # Process each filter
    for f, cols in filter_columns.items():
        # Filter by this specific filter
        f_mask = (sn_data['Filter'] == f) if 'Filter' in sn_data.columns else True
        f_data = sn_data[f_mask].copy()

        if len(f_data) < 5:
            continue

        # Get JD, magnitude, error columns
        jd = f_data['MJD'].values  # CFA3 uses MJD

        # Try filter-specific columns first, fallback to generic
        mag_col = cols['mag'] if cols['mag'] in f_data.columns else 'STDMAG'
        err_col = cols['err'] if cols['err'] in f_data.columns else 'dMAG'

        if mag_col not in f_data.columns or err_col not in f_data.columns:
            continue

        mag = f_data[mag_col].dropna().values
        mag_err = f_data[err_col].fillna(0.05).dropna().values  # Default 0.05

        # Align arrays (remove NaNs)
        valid_idx = ~(np.isnan(jd) | np.isnan(mag) | np.isnan(mag_err))
        jd, mag, mag_err = jd[valid_idx], mag[valid_idx], mag_err[valid_idx]

        if len(jd) < 5:
            continue

        # Sort chronologically
        sort_idx = np.argsort(jd)
        jd, mag, mag_err = jd[sort_idx], mag[sort_idx], mag_err[sort_idx]

        # YOUR EXACT PEAK FINDING ALGORITHM
        m_max, m_err = extract_peak_magnitude(jd, mag, mag_err)

        # Store in original row
        row_idx = short_df[short_df.iloc[:, 0].astype(str) == short_sn].index[0]
        results_df.at[row_idx, f'{f}_m_max'] = m_max
        results_df.at[row_idx, f'{f}_m_max_err'] = m_err

        if not np.isnan(m_max):
            print(f"  ✅ {f}: m_max = {m_max:.3f} ± {m_err:.3f}")

print(f"\n🎉 PROCESSING COMPLETE!")
print(f"✅ Successfully matched {successful_matches}/{len(sn_names)} SNe")
results_df.to_excel(output_path, index=False)
print(f"✅ SAVED: {output_path}")

# =============================================================================
# SUMMARY TABLE
# =============================================================================
print("\n📊 PEAK MAGNITUDE SUMMARY (first 10 SNe):")
summary_cols = [short_df.columns[0]] + [f'{f}_m_max' for f in filter_columns.keys()]
print(results_df[summary_cols].round(3).head(10))

print(f"\n✅ **NEW COLUMNS ADDED**:")
print(f"   U_m_max, U_m_max_err")
print(f"   B_m_max, B_m_max_err")
print(f"   V_m_max, V_m_max_err")
print(f"   R_m_max, R_m_max_err")
print(f"   I_m_max, I_m_max_err")
print(f"\n✅ Ready for GNN/Phillips relation analysis!")

print("\nFirst 10 rows of the updated DataFrame:")
print(results_df.head(10))
print("\nColumn names and data types of the updated DataFrame:")
print(results_df.info())

🔄 Loading datasets...
✅ Shortlisted: 41 SNe
✅ CFA3: 11499 data points
CFA3 columns: ['SN', 'MJD', 'Filter', 'STDMAG', 'dMAG']
First 5 shortlisted SNe: ['1986G', '1994D', '1997dt', '1998aq', '1998bu']

🚀 Extracting peak magnitudes for ALL FILTERS...

[1/41] Processing: 1986G
  ❌ No CFA3 match

[2/41] Processing: 1994D
  ❌ No CFA3 match

[3/41] Processing: 1997dt
  ❌ No CFA3 match

[4/41] Processing: 1998aq
  ❌ No CFA3 match

[5/41] Processing: 1998bu
  ❌ No CFA3 match

[6/41] Processing: 1998dh
  ❌ No CFA3 match

[7/41] Processing: 1999cl
  ❌ No CFA3 match

[8/41] Processing: 1999ee
  ❌ No CFA3 match

[9/41] Processing: 2001cp
  ✅ MATCHED: 2001cp → SN01CP
  📊 10 data points

[10/41] Processing: 2002bo
  ✅ MATCHED: 2002bo → SN02BO
  📊 86 data points
  ✅ U: m_max = 13.973 ± 0.056
  ✅ B: m_max = 14.045 ± 0.018
  ✅ V: m_max = 13.634 ± 0.013
  ✅ R: m_max = 13.499 ± 0.013
  ✅ I: m_max = 13.611 ± 0.011

[11/41] Processing: 2002cr
  ✅ MATCHED: 2002cr → SN02CR
  📊 44 data points
  ✅ U: m_max = 1

The below code is to extract the extinction value, distance modulus and modulus error from the .dat file obtained from the website: "https://github.com/PantheonPlusSH0ES/DataRelease/blob/main/Pantheon%2B_Data/4_DISTANCES_AND_COVAR/Pantheon%2BSH0ES.dat" and add it to the file containing the 185 supernovae...

In [ ]:
import pandas as pd
import numpy as np

# File paths - UPDATE THESE PATHS FOR YOUR ENVIRONMENT
# Since Pantheon-SH0ES.dat is attached as 'Pantheon-SH0ES.dat', use that directly
pantheon_dat_path = "/content/drive/MyDrive/apparent magnitude data/Pantheon+SH0ES.dat"
cfa3_path = "/content/drive/MyDrive/apparent magnitude data/cfa3_lightcurves (1).xlsx"  # Update if needed

def convert_sn_name(sn_name):
    """Convert 'sn01C' → '2001C' or handle variations"""
    if pd.isna(sn_name) or sn_name == 'SN':
        return None
    sn_str = str(sn_name).strip().upper()
    if sn_str.startswith('SN'):
        year = sn_str[2:4]
        letter = sn_str[4:]
        return f"20{year}{letter}"
    return sn_str

# Load CfA3 data
print("🔍 Loading CfA3 file...")
cfa3_df = pd.read_excel(cfa3_path)
cfa3_sns = cfa3_df['SN'].dropna().unique()
print(f"Found {len(cfa3_sns)} unique CfA3 SNe: {cfa3_sns[:5]}")

# Load Pantheon+SH0ES DAT file PROPERLY
print("\n🔍 Loading Pantheon+SH0ES.dat...")
pantheon_raw = pd.read_csv(pantheon_dat_path, sep=r'\s+', comment='#', header=None)

# First row is HEADER - extract column names and indices
header_row = pantheon_raw.iloc[0]
data_rows = pantheon_raw.iloc[1:].reset_index(drop=True)

# Find exact column indices using header names
try:
    cid_col = header_row[header_row.str.contains('CID', case=False, na=False)].index[0]
    mu_col = header_row[header_row.str.contains('MU_SH0ES', case=False, na=False)].index[0]
    mu_err_col = header_row[header_row.str.contains('MU_SH0ES_ERR_DIAG|MU_SH0ESERRDIAG', case=False, na=False)].index[0]
    mwebv_col = header_row[header_row.str.contains('MWEBV', case=False, na=False)].index[0]

    print(f"✅ Column indices found:")
    print(f"   CID: {cid_col}, MU_SH0ES: {mu_col}, MU_ERR: {mu_err_col}, MWEBV: {mwebv_col}")
    print(f"   Total columns: {len(header_row)}")

except:
    print("❌ Could not find column headers - check file format")
    print("Header sample:", header_row[:15].tolist())
    exit()

pantheon_sn_col = int(cid_col)
pantheon_mu_col = int(mu_col)
pantheon_mu_err_col = int(mu_err_col)
pantheon_mwebv_col = int(mwebv_col)

# Process matches
matches = []
print(f"\n🔍 Searching for matches among {len(data_rows)} Pantheon rows...")

for cfa_sn in cfa3_sns:
    pantheon_sn = convert_sn_name(cfa_sn)

    # Search in SN name column (case-insensitive)
    mask = data_rows.iloc[:, pantheon_sn_col].astype(str).str.contains(
        pantheon_sn, case=False, na=False
    )
    found_rows = data_rows[mask]

    if not found_rows.empty:
        # Take FIRST matching row (as in original code)
        row = found_rows.iloc[0]

        # Extract values SAFELY
        mu_sh0es = pd.to_numeric(row.iloc[pantheon_mu_col], errors='coerce')
        mu_sh0es_err = pd.to_numeric(row.iloc[pantheon_mu_err_col], errors='coerce')
        mwebv = pd.to_numeric(row.iloc[pantheon_mwebv_col], errors='coerce')

        # Filter invalid values
        if mu_sh0es == -999.0 or pd.isna(mu_sh0es) or mu_sh0es > 100:
            mu_sh0es = np.nan
        if pd.isna(mu_sh0es_err) or mu_sh0es_err <= 0:
            mu_sh0es_err = np.nan
        if pd.isna(mwebv) or mwebv == -999.0:
            mwebv = np.nan

        matches.append({
            'cfa3_name': cfa_sn,
            'pantheon_name': pantheon_sn,
            'pantheon_row_idx': found_rows.index[0],  # Original row index
            'MU_SH0ES': mu_sh0es,
            'MU_SH0ES_ERR_DIAG': mu_sh0es_err,
            'MWEBV': mwebv
        })
        print(f"   ✅ Match: {cfa_sn} → {pantheon_sn} (MU={mu_sh0es:.3f})")

matches_df = pd.DataFrame(matches)
print(f"\n🎉 Found {len(matches_df)} total matches!")

# Display match statistics
print("\n📊 Match Summary:")
print(matches_df[['cfa3_name', 'pantheon_name', 'MU_SH0ES', 'MU_SH0ES_ERR_DIAG', 'MWEBV']].head(10))

# Merge with CfA3 data
if len(matches_df) > 0:
    # Create dictionaries for mapping
    mu_dict = dict(zip(matches_df['cfa3_name'], matches_df['MU_SH0ES']))
    mu_err_dict = dict(zip(matches_df['cfa3_name'], matches_df['MU_SH0ES_ERR_DIAG']))
    mwebv_dict = dict(zip(matches_df['cfa3_name'], matches_df['MWEBV']))

    # Add to CfA3 dataframe
    cfa3_df['MU_SH0ES'] = cfa3_df['SN'].map(mu_dict)
    cfa3_df['MU_SH0ES_ERR_DIAG'] = cfa3_df['SN'].map(mu_err_dict)
    cfa3_df['MWEBV'] = cfa3_df['SN'].map(mwebv_dict)

    # Verify
    matched_mask = cfa3_df['MU_SH0ES'].notna()
    print(f"\n✅ Added values to {matched_mask.sum()} CfA3 rows ({matched_mask.sum()/len(cfa3_df)*100:.1f}% match rate)")

    print("\n📋 Sample enhanced rows:")
    sample = cfa3_df[matched_mask][['SN', 'MU_SH0ES', 'MU_SH0ES_ERR_DIAG', 'MWEBV']].head()
    print(sample)

    # Save enhanced files
    output_path = "/content/drive/MyDrive/apparent magnitude data/cfa3_lightcurves_enhanced.xlsx"
    cfa3_df.to_excel(output_path, index=False)
    print(f"\n💾 Enhanced CfA3 saved: {output_path}")

    matches_path = "/content/drive/MyDrive/apparent magnitude data/cfa3_pantheon_matches_detailed.csv"
    matches_df.to_csv(matches_path, index=False)
    print(f"💾 Matches summary saved: {matches_path}")

else:
    print("❌ No matches found. Check SN name formats.")
    print("CfA3 sample names:", cfa3_sns[:10])
    print("Pantheon sample names:", data_rows.iloc[:, pantheon_sn_col].unique()[:10])

print("\n🎯 Ready for absolute magnitude: M = m - MU_SH0ES - (3.1 * MWEBV)")
print("✅ Fixed: Proper header parsing, dynamic column detection, safe numeric conversion!")


🔍 Loading CfA3 file...
Found 185 unique CfA3 SNe: ['sn01C' 'sn01G' 'sn01N' 'sn01V' 'sn01ah']

🔍 Loading Pantheon+SH0ES.dat...
✅ Column indices found:
   CID: 0, MU_SH0ES: 10, MU_ERR: 11, MWEBV: 33
   Total columns: 47

🔍 Searching for matches among 1701 Pantheon rows...
   ✅ Match: sn01C → 2001C (MU=33.878)
   ✅ Match: sn01V → 2001V (MU=33.790)
   ✅ Match: sn01ah → 2001AH (MU=36.980)
   ✅ Match: sn01az → 2001AZ (MU=36.315)
   ✅ Match: sn01bf → 2001BF (MU=33.993)
   ✅ Match: sn01cp → 2001CP (MU=34.965)
   ✅ Match: sn01da → 2001DA (MU=34.022)
   ✅ Match: sn01eh → 2001EH (MU=36.007)
   ✅ Match: sn01en → 2001EN (MU=33.999)
   ✅ Match: sn01ep → 2001EP (MU=33.636)
   ✅ Match: sn01fe → 2001FE (MU=33.892)
   ✅ Match: sn02G → 2002G (MU=35.806)
   ✅ Match: sn02bf → 2002BF (MU=34.944)
   ✅ Match: sn02cr → 2002CR (MU=33.231)
   ✅ Match: sn02de → 2002DE (MU=35.364)
   ✅ Match: sn02dj → 2002DJ (MU=32.901)
   ✅ Match: sn02dp → 2002DP (MU=33.381)
   ✅ Match: sn02eu → 2002EU (MU=36.062)
   ✅ Match: sn0

In [ ]:
!pip install extinction


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.8/627.8 kB 8.6 MB/s eta 0:00:00


The below code will Take the extinction values, distance modulus and modulus error for each supernovae and run the photometory code for the estimation of apparent and absolute magnitude and also the delta m15 value.....

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import UnivariateSpline
from astropy.stats import sigma_clipped_stats
import matplotlib.pyplot as plt
from tqdm import tqdm

# Load your ENHANCED dataset
input_path = "/content/drive/MyDrive/apparent magnitude data/cfa3_data_table.xlsx"
df = pd.read_excel(input_path)

print("Dataset shape:", df.shape)
print("Distance columns found:", [col for col in df.columns if 'MU' in col.upper()])

# Standard Cardelli 1989 extinction coefficients (R_V = 3.1)
ext_coeffs = {'U': 1.664, 'B': 1.321, 'V': 1.000, 'R': 0.782, 'I': 0.601}

def fit_single_filter(jd, mag, mag_err, filter_name):
    """ROBUST peak fitting with outlier rejection"""
    try:
        # Filter good data points
        mask = (~np.isnan(jd)) & (~np.isnan(mag)) & (~np.isnan(mag_err)) & (mag_err < 0.5)
        if np.sum(mask) < 8:
            return np.nan, np.nan

        jd_clean, mag_clean, mag_err_clean = jd[mask], mag[mask], mag_err[mask]
        n_points = len(jd_clean)
        s_smooth = max(0.1 * n_points, 0.5)

        # *** OUTLIER REJECTION: Sigma clip input data first ***
        mag_mean, mag_median, mag_std = sigma_clipped_stats(mag_clean)
        input_mask = np.abs(mag_clean - mag_median) < 3 * mag_std
        jd_clean = jd_clean[input_mask]
        mag_clean = mag_clean[input_mask]
        mag_err_clean = mag_err_clean[input_mask]

        if len(jd_clean) < 6:  # Need minimum after cleaning
            return np.nan, np.nan

        # Smoother data
        mag_smooth = np.convolve(mag_clean, np.ones(5)/5, mode='same')
        jd_range = jd_clean.max() - jd_clean.min()
        fine_jd = np.linspace(jd_clean.min() - 0.1*jd_range,
                             jd_clean.max() + 0.1*jd_range, 2000)

        spline = UnivariateSpline(jd_clean, mag_smooth, k=3, s=s_smooth)
        fine_mag = spline(fine_jd)

        central_mask = (fine_jd > np.percentile(fine_jd, 15)) & \
                      (fine_jd < np.percentile(fine_jd, 85))
        if np.sum(central_mask) < 100:
            return np.nan, np.nan

        peak_idx = np.argmin(fine_mag[central_mask])
        mag_peak = fine_mag[central_mask][peak_idx]

        # *** PHYSICAL REALISM CHECK ***
        if mag_peak < 5 or mag_peak > 25:  # Impossible peak magnitudes
            return np.nan, np.nan

        # Monte Carlo
        n_iter = 3000
        mag_peak_sim = []

        for _ in range(n_iter):
            mag_sim = mag_clean + np.random.normal(0, mag_err_clean)
            try:
                spline_sim = UnivariateSpline(jd_clean, mag_sim, k=3, s=s_smooth)
                fine_mag_sim = spline_sim(fine_jd)
                peak_idx_sim = np.argmin(fine_mag_sim[central_mask])
                sim_peak = fine_mag_sim[central_mask][peak_idx_sim]

                # Reject unrealistic simulations
                if 5 < sim_peak < 25:
                    mag_peak_sim.append(sim_peak)
            except:
                continue

        if len(mag_peak_sim) < 20:
            return np.nan, np.nan

        mag_peak_sim = np.array(mag_peak_sim)
        mag_peak_mean, _, mag_peak_std = sigma_clipped_stats(mag_peak_sim)

        # *** FINAL VALIDATION ***
        if mag_peak_mean < 5 or mag_peak_mean > 25 or mag_peak_std > 2.0:
            return np.nan, np.nan

        return mag_peak_mean, mag_peak_std

    except:
        return np.nan, np.nan

def fit_b_delta_m15(jd, mag, mag_err):
    """Robust Δm15 for B-band"""
    try:
        mask = (~np.isnan(jd)) & (~np.isnan(mag)) & (~np.isnan(mag_err)) & (mag_err < 0.5)
        if np.sum(mask) < 8:
            return np.nan, np.nan

        jd_clean, mag_clean, mag_err_clean = jd[mask], mag[mask], mag_err[mask]

        # Sigma clip input
        mag_mean, mag_median, mag_std = sigma_clipped_stats(mag_clean)
        input_mask = np.abs(mag_clean - mag_median) < 3 * mag_std
        jd_clean = jd_clean[input_mask]
        mag_clean = mag_clean[input_mask]
        mag_err_clean = mag_err_clean[input_mask]

        if len(jd_clean) < 6:
            return np.nan, np.nan

        n_points = len(jd_clean)
        s_smooth = max(0.1 * n_points, 0.5)

        mag_smooth = np.convolve(mag_clean, np.ones(5)/5, mode='same')
        jd_range = jd_clean.max() - jd_clean.min()
        fine_jd = np.linspace(jd_clean.min() - 0.1*jd_range,
                             jd_clean.max() + 0.1*jd_range, 2000)

        spline = UnivariateSpline(jd_clean, mag_smooth, k=3, s=s_smooth)
        fine_mag = spline(fine_jd)

        central_mask = (fine_jd > np.percentile(fine_jd, 15)) & \
                      (fine_jd < np.percentile(fine_jd, 85))
        peak_idx = np.argmin(fine_mag[central_mask])
        jd_peak = fine_jd[central_mask][peak_idx]

        dm15 = spline(jd_peak + 15) - np.min(fine_mag[central_mask])

        # Physical check
        if dm15 < 0.5 or dm15 > 3.0:  # Impossible decline rates
            return np.nan, np.nan

        n_iter = 3000
        dm15_sim = []
        for _ in range(n_iter):
            mag_sim = mag_clean + np.random.normal(0, mag_err_clean)
            try:
                spline_sim = UnivariateSpline(jd_clean, mag_sim, k=3, s=s_smooth)
                fine_mag_sim = spline_sim(fine_jd)
                peak_idx_sim = np.argmin(fine_mag_sim[central_mask])
                jd_peak_sim = fine_jd[central_mask][peak_idx_sim]
                sim_dm15 = spline_sim(jd_peak_sim + 15) - np.min(fine_mag_sim[central_mask])

                if 0.5 < sim_dm15 < 3.0:
                    dm15_sim.append(sim_dm15)
            except:
                continue

        if len(dm15_sim) < 20:
            return np.nan, np.nan

        dm15_sim = np.array(dm15_sim)
        dm15_mean, _, dm15_std = sigma_clipped_stats(dm15_sim)

        if dm15_mean < 0.5 or dm15_mean > 3.0 or dm15_std > 0.5:
            return np.nan, np.nan

        return dm15_mean, dm15_std

    except:
        return np.nan, np.nan

# Filter SNe with SH0ES distances
valid_sne = df.dropna(subset=['MU_SH0ES', 'MU_SH0ES_ERR_DIAG'])['SN'].unique()
print(f"\n✅ Found {len(valid_sne)} SNe with SH0ES distances")

# Process ALL SNe (remove [:50] limit)
results = []
rejected_count = {'U':0, 'B':0, 'V':0, 'R':0, 'I':0, 'dm15':0}

for sn in tqdm(valid_sne, desc="🔥 Processing ALL SNe"):
    sn_data = df[df['SN'] == sn].copy()

    mu = sn_data['MU_SH0ES'].iloc[0]
    mu_err = sn_data['MU_SH0ES_ERR_DIAG'].iloc[0]
    mwebv = sn_data.get('MWEBV', pd.Series([0.0])).iloc[0]

    sn_results = {
        'SN': sn, 'MU_SH0ES': mu, 'MU_ERR': mu_err, 'MWEBV': mwebv
    }

    for filter_name in ['U', 'B', 'V', 'R', 'I']:
        filter_data = sn_data[sn_data['Filter'] == filter_name]

        if len(filter_data) >= 8:
            jd = filter_data['MJD'].values
            mag = filter_data['STDMAG'].values
            mag_err = filter_data['dMAG'].values

            app_peak, app_peak_err = fit_single_filter(jd, mag, mag_err, filter_name)

            if not np.isnan(app_peak):
                A_lambda = ext_coeffs.get(filter_name, 1.0) * abs(mwebv)
                abs_mag = app_peak - mu - A_lambda

                # *** ABSOLUTE MAG OUTLIER REJECTION ***
                if abs_mag < -25 or abs_mag > -15:  # Type Ia range
                    rejected_count[filter_name] += 1
                else:
                    sn_results.update({
                        f'{filter_name}_app_peak': app_peak,
                        f'{filter_name}_app_peak_err': app_peak_err,
                        f'{filter_name}_abs_mag': abs_mag,
                        f'{filter_name}_abs_mag_err': np.sqrt(app_peak_err**2 + mu_err**2)
                    })

    # B-band Δm15
    b_data = sn_data[sn_data['Filter'] == 'B']
    if len(b_data) >= 8:
        jd_b = b_data['MJD'].values
        mag_b = b_data['STDMAG'].values
        mag_err_b = b_data['dMAG'].values

        dm15_b, dm15_err_b = fit_b_delta_m15(jd_b, mag_b, mag_err_b)
        if not np.isnan(dm15_b):
            sn_results['B_dm15'] = dm15_b
            sn_results['B_dm15_err'] = dm15_err_b
        else:
            rejected_count['dm15'] += 1

    results.append(sn_results)

# Results
results_df = pd.DataFrame(results)
results_df.to_excel('cfa3_ubvri_cleaned.xlsx', index=False)

print(f"\n🎉 SAVED {len(results_df)} SNe to cfa3_ubvri_cleaned.xlsx")
print(f"Rejected: U={rejected_count['U']}, B={rejected_count['B']}, V={rejected_count['V']}, R={rejected_count['R']}, I={rejected_count['I']}, dm15={rejected_count['dm15']}")

# CLEAN VALIDATION
print("\n📊 CLEANED UBVRI RESULTS:")
for band in ['U', 'B', 'V', 'R', 'I']:
    abs_col = f'{band}_abs_mag'
    if abs_col in results_df.columns:
        valid = results_df[abs_col].dropna()
        if len(valid) > 0:
            print(f"{band}: {len(valid)} SNe, mean={valid.mean():.3f}±{valid.std():.3f} (range: {valid.min():.2f} to {valid.max():.2f})")

print("\n📊 CLEANED B-BAND Δm15:")
b_dm15 = results_df['B_dm15'].dropna()
if len(b_dm15) > 0:
    print(f"Δm15(B): {len(b_dm15)} SNe, mean={b_dm15.mean():.3f}±{b_dm15.std():.3f}")

print("\n✅ OUTLIERS REMOVED: Only physical Type Ia magnitudes kept!")


Dataset shape: (11499, 8)
Distance columns found: ['MU_SH0ES', 'MU_SH0ES_ERR_DIAG']

✅ Found 112 SNe with SH0ES distances


🔥 Processing ALL SNe:  15%|█▌        | 17/112 [00:14<01:06,  1.42it/s]/tmp/ipython-input-66375940.py:46: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  spline = UnivariateSpline(jd_clean, mag_smooth, k=3, s=s_smooth)
🔥 Processing ALL SNe:  54%|█████▍    | 61/112 [00:48<00:53,  1.04s/it]/tmp/ipython-input-66375940.py:68: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  spline_sim = UnivariateSpline(jd_clean, mag_sim, k=3, s=s_smooth)
🔥 Processing ALL SNe: 100%|██████████| 112/112 [01:24


🎉 SAVED 112 SNe to cfa3_ubvri_cleaned.xlsx
Rejected: U=0, B=1, V=1, R=0, I=0, dm15=57

📊 CLEANED UBVRI RESULTS:
U: 24 SNe, mean=-18.584±0.767 (range: -19.52 to -16.29)
B: 55 SNe, mean=-18.275±0.687 (range: -19.27 to -15.56)
V: 66 SNe, mean=-18.535±0.524 (range: -19.31 to -16.00)
R: 19 SNe, mean=-18.836±0.254 (range: -19.25 to -18.34)
I: 14 SNe, mean=-18.584±0.205 (range: -18.86 to -18.24)

📊 CLEANED B-BAND Δm15:
Δm15(B): 22 SNe, mean=1.098±0.354

✅ OUTLIERS REMOVED: Only physical Type Ia magnitudes kept!
